In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## BOX
# This is the dimensions of the unit cell
(like transformation matrix in vesta but the entries are actual lengths, not units 1, 2, 8, etc)

In [3]:
#creating a dataframe from "box-filename.txt"
#this .txt has the filenames for the unit cell dimensions of each unit cell
#
filenames = pd.read_csv('box-filename.txt',header=None)
filenames.head()
#
#need to find out again what c11 and box 104 etc means
#

FileNotFoundError: [Errno 2] No such file or directory: 'box-filename.txt'

In [ ]:
#turning column 0 in filenames into a list
#there is only one collumn here so it's already kinda a list but important for later manipulation of df
#and also important if the df had more than one column
#
filenames = list(filenames[0])

In [ ]:
#create an empty list called boxes
#iterates through box numbers
#turns 3x3 matrix for unit cell parameters into a 1x9 object
boxes = []
for name in filenames:
    cell = np.loadtxt('box/' + str(name), delimiter=' ').reshape(1,9)
    boxes.append(cell[0])
    
print(boxes[19])
#as far as I'm concerned it should be [x, 0, 0,      0, y, 0,     0, 0, z]

In [ ]:
#this reads off "box-filenames.txt" and then removes the .csv tag (with [:-4]) to create indexes for a dataframe
ind = [str(name[:-4])for name in filenames]
ind[0]

In [ ]:
#3x3 matrix transformed into 1x9,
#this keeps track of them
col = ['xx','xy','xz','yx','yy','yz','zx','zy','zz']

In [ ]:
#checking how long the df should be (where do these come from again??)
77+320+362+158+210+316+237

In [ ]:
#finally creating the dataframe
df_box = pd.DataFrame(boxes,index=ind, columns=col)
df_box
#why are the xz and zx values not zero?

## POT
### This is the pseudopotentials

In [ ]:
#the feature vectors are created based off 11 parameters mentioned in the pseudopotential
pot = pd.read_csv('potcar_param.csv')
pot

In [ ]:
#transpose
pot.T

In [ ]:
#removes collumn 1,2,3,4,9,10 (why?)
table = pot.drop([1,2,3,4,9,10],axis=0).T
table

#### table 2

In [ ]:
#creating an array for every row in the table abobe
pot_name= np.array(pot.iloc[:,0])
pot_Fe= np.array(pot.iloc[:,1])
pot_O= np.array(pot.iloc[:,2])
pot_Ti= np.array(pot.iloc[:,3])
pot_Bi= np.array(pot.iloc[:,4])

pot_name

In [ ]:
pot_Fe

In [ ]:
len(pot_Bi),len(pot_Ti)

### coooord test

In [ ]:
#test, reading the first file which contains coords of all the atoms
bx = np.loadtxt('coord/c1-coord1',skiprows=1)
bx

In [ ]:
pd.DataFrame(bx).to_csv

## COORD

In [ ]:
#same as what was done with the unit cell, but this time with atom coordinates
filenames = pd.read_csv('coord-filename.txt',header=None)

In [ ]:
filenames = list(filenames[0])

In [ ]:
#just an example of what is going on
filenames[0],filenames[320],filenames[682],filenames[840],filenames[1050],filenames[1127],filenames[1443]

In [ ]:
#again reshaping into one dimension as machine learning requires input to be 1 dimensional
coords = []
for name in filenames:
    c = np.loadtxt('coord/' + str(name), delimiter=' ').reshape(1,348)
    coords.append(c[0])

In [ ]:
len(coords)

In [ ]:
coords[0]

## coord pot

In [ ]:
coords[0][:72]

In [ ]:
#the first 24 atoms are bismuth, so making list of bismuth positions for first coords file (24*3=72)
Bi_xyz = coords[0][:72]

In [ ]:
#pot that has been made previously
pot_Bi

In [ ]:
#multiply each position by the potential to create an array for each single bismuth position value (the decoupling thing)
Bi_coord_pot = []
for i in Bi_xyz:
    Bi_coord_pot.append(i*pot_Bi)

In [ ]:
#should be same no of arrays as there are xyz values for bismuth position
len(Bi_coord_pot)

In [ ]:
#again, everything needs to be in one dimension
np.array(Bi_coord_pot).reshape(1,792)

In [ ]:
#based on the example above, creating the decoupled arrays for all 1680 different configurations
Bi_all = []
for i in range(0,1680):
    Bi_xyz = coords[i][:72]
    Bi_coord_pot = []
    for i in Bi_xyz:
        Bi_coord_pot.append(i*pot_Bi)
    Bi_all.append(np.array(Bi_coord_pot).reshape(792))

In [ ]:
len(Bi_all)

In [ ]:
#doing the same for Ti, Fe below
Ti_all = []
for i in range(0,1680):
    Ti_xyz = coords[i][72:108]
    Ti_coord_pot = []
    for i in Ti_xyz:
        Ti_coord_pot.append(i*pot_Ti)
    Ti_all.append(np.array(Ti_coord_pot).reshape(396))

In [ ]:
Ti_all[35]

In [ ]:
Fe_all = []
for i in range(0,1680):
    Fe_xyz = coords[i][324:348]
    Fe_coord_pot = []
    for i in Fe_xyz:
        Fe_coord_pot.append(i*pot_Fe)
    Fe_all.append(np.array(Fe_coord_pot).reshape(264))

In [ ]:
len(Fe_all[0])

In [ ]:
df_Bi = pd.DataFrame(Bi_all)
df_Bi

In [ ]:
pd.DataFrame(Ti_all)

In [ ]:
pd.DataFrame(Fe_all)

In [ ]:
Ti_Fe = pd.concat([pd.DataFrame(Ti_all),pd.DataFrame(Fe_all)], axis=1, ignore_index=True)
Ti_Fe

In [ ]:
#df with all the coordinates concated together
Bi_Ti_Fe = pd.concat([pd.DataFrame(Bi_all),Ti_Fe], axis=1, ignore_index=True)
Bi_Ti_Fe

In [ ]:
#adding the unit cell dimesions to the front of the coordinates df
box_Bi_Ti_Fe = pd.concat([pd.DataFrame(boxes),Bi_Ti_Fe], axis=1, ignore_index=True)
box_Bi_Ti_Fe

In [ ]:
#box_Ti_Fe = pd.concat([pd.DataFrame(boxes),Ti_Fe], axis=1, ignore_index=True)
#box_Ti_Fe

## energy

In [ ]:
#enegies for each configuration
energy_df = pd.read_csv('oszicar/energys.csv',header=None, sep = ' ')

In [ ]:
energy_df.shape

In [ ]:
energy_df

In [ ]:
#adding the energies to position dataframe (at the end as it is the target)
df = pd.concat([box_Bi_Ti_Fe,energy_df[1]], axis=1, ignore_index=True)
df

In [ ]:
df.describe()

## Clustering
#### clustering all the different configurations together

In [ ]:
from sklearn.cluster import KMeans as km

In [ ]:
km(n_clusters=7,random_state=1).fit_transform(df)

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
xy = PCA(n_components=2).fit_transform(km(n_clusters=7,random_state=1).fit_transform(df))
xy[:,0]

In [ ]:
plt.plot(0.02*xy[:,0],0.02*xy[:,1], '.')#, label = labels)
#plt.xlim(-80, -75)
#plt.legend(plot_labels)

In [ ]:
#I believe that this implies there are 7 distinct phases/structures/somethinglikethat for all 1680 configurations?

## prepare data set

In [ ]:
#1680 from 1680 configs and 1462 from atom positions and energies?
df.shape

In [ ]:
#train_val_test split, deciding how big each category should be
valid_samples = int(round(.2*df.shape[0]))
test_samples = int(round(.1*df.shape[0]))
train_samples = df.shape[0]-test_samples-valid_samples

In [ ]:
train_samples, valid_samples, test_samples

In [ ]:
#df.index.tolist()

In [ ]:
# Set a random seed
RNG_SEED = 40
np.random.seed(seed=RNG_SEED)

#creating test/train/val split based on sizes above
#randomly add to val, then remove all from all_f if in val
#then randomly add to test from what is left, everything in test removed from all_f
#all thats left is testing data
#note this is just the indexes, being randomly assigned, can then go back to df using indices and create actual dfs
all_f = df.index.tolist().copy()
valid_data = np.random.choice(all_f,
                              size=valid_samples,
                              replace=False)


all_f = [ f for f in all_f if f not in valid_data]
test_data = np.random.choice(all_f,
                              size=test_samples,
                              replace=False)

all_f = [ f for f in all_f if f not in test_data]
train_data = all_f.copy()

In [ ]:
#showing that some entries are missing from train when compared to all (indexes don't match)
train_data[10]

In [ ]:
#df.isin(test_data)

In [ ]:
#creating the train val test split and checking they are the correct size
cp_train = df.loc[train_data]
cp_train.shape

In [ ]:
cp_valid = df.loc[valid_data]
cp_valid.shape

In [ ]:
cp_test = df.loc[test_data]
cp_test.shape

In [ ]:
#sampling the split
cp_train_sample = cp_train.sample(n=1100, random_state=RNG_SEED)
cp_valid_sample = cp_valid.sample(n=150, random_state=RNG_SEED)
cp_test_sample = cp_test.sample(n=150, random_state=RNG_SEED)

len(cp_train_sample),len(cp_valid_sample),len(cp_test_sample)

In [ ]:
cp_train_sample

In [ ]:
cp_train_sample.iloc[:,:-1]

In [ ]:
#final variables with split made (no energy here)
y_train = cp_train_sample.iloc[:,1461]
y_valid = cp_valid_sample.iloc[:,1461]
y_test = cp_test_sample.iloc[:,1461]

In [ ]:
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [ ]:
#scaling and normalising the data
#.iloc[:,:-1] removes the energy values from the scaling
#energy is the target
X_train = scaler.fit_transform(cp_train_sample.iloc[:,:-1])
X_valid = scaler.transform(cp_valid_sample.iloc[:,:-1])
X_test  = scaler.transform(cp_test_sample.iloc[:,:-1])

In [ ]:
X_train = normalize(X_train)
X_valid = normalize(X_valid)
X_test = normalize(X_test)

## TRAINING

In [ ]:
from time import time

### methods
from sklearn.dummy import DummyRegressor

from sklearn.linear_model import Ridge

from sklearn.neighbors import KNeighborsRegressor

from sklearn.svm import SVR
from sklearn.svm import LinearSVR

from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor

### metrics

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [ ]:
nine_methods = [DummyRegressor,
                Ridge,
                KNeighborsRegressor,
                SVR,
                LinearSVR,
                AdaBoostRegressor,
                GradientBoostingRegressor,
                ExtraTreesRegressor,
                RandomForestRegressor]

In [ ]:
nine_methods[1]

In [ ]:
### Measurements

def meas_method (act, pred):
    r2   = r2_score(act, pred)
    mae  = mean_absolute_error(act,pred)
    rmse = mean_squared_error(act,pred,squared=False)
    
    return r2, mae, rmse

In [ ]:
###Evaluate
#intialises method, starts a timer, trains the models, timer finishes, time taken to train found
#accuracy on training validation set found

def evaluate (X_train, y_train, X_valid, y_valid, methods):
    
    fit_time   = []
    meas_train = []
    meas_valid = []
    pred_valid = []
    for m in methods:
    
        method =m()
        start = time()
        method.fit(X_train, y_train)
        end = time()
        delta = end-start
        print(m.__name__, '{:0.4f}'.format(delta)+'s')
        y_pred_t = method.predict(X_train)
        y_pred_v = method.predict(X_valid)
        
        fit_time.append(delta)
        pred_valid.append(y_pred_v)
        meas_train.append(meas_method(y_train,y_pred_t))
        meas_valid.append(meas_method(y_valid,y_pred_v))
    
    return meas_train, meas_valid, pred_valid, fit_time

In [ ]:
e = evaluate(X_train,y_train,X_valid, y_valid,nine_methods)

In [ ]:
###Define plot + regression y_pred
#function to make a plot of pred vs actual energy for each model
def plot_method (act, pred, method):
    xy_max = np.max([np.max(act), np.max(pred)])
    xy_min = np.min([np.min(act), np.min(pred)])
    polyfit = np.polyfit(act,pred,deg=1)
    reg_lsp = np.poly1d(polyfit)
    
    #print (reg_lsp, reg_lsp[1])
    
    line = [ i+.5 for i in range(int(xy_min)-1,int(xy_max)+1,2)]
    reg_lsm = reg_lsp(line)
    
    plt.plot(act,pred, 'o', ms=9, mec= 'k', mfc= 'silver', alpha=.4)
    plt.plot([xy_min,xy_max],[xy_min,xy_max],'k--', label='Target')
    plt.plot(line,reg_lsm,'r-', label ='Linear fit')#+str(reg_lsp))   
    
    #plt.
   # plt.xlim(xy_min-.2,xy_max+.2)
    plt.xlabel(r'Actual Energy',fontsize=16)
    plt.ylabel(r'Predicted Energy',fontsize=16)
    plt.legend(loc='upper left',fontsize=14)
    plt.title(str(method.__name__)+
              ', score='+ '{:0.4f}'.format(r2_score(act,pred))
              ,fontsize=20)
    #plt.show()
    return f'{reg_lsp[1]:0.6f}'

In [ ]:
np.max([np.max(y_valid),np.max(e[2][2])])

In [ ]:
e_five = plot_method (y_valid,e[2][2],nine_methods[2])

In [ ]:
e_five

In [ ]:
meas_train = e[0]
meas_valid = e[1]
time_fitti = e[3]

In [ ]:
#creating mini datafames for all the different values
#then putting them all together to see how the models perform
df_time = pd.DataFrame(time_fitti, columns=['Fitting time'],
                       index=[m.__name__ for m in nine_methods])

df_train = pd.DataFrame(meas_train, 
                        columns=['r2_score_train', 'mae_train', 'rmse_train'],
                        index=[m.__name__ for m in nine_methods])

df_coef = pd.DataFrame(e_five, columns=['coef.lin_valid'],
                       index=[m.__name__ for m in nine_methods])

df_valid = pd.DataFrame(meas_valid, 
                   columns=['r2_score_valid', 'mae_valid', 'rmse_valid'], 
                   index=[m.__name__ for m in nine_methods])

df = pd.concat([df_time,df_train,df_valid,df_coef], axis=1)
df = df.sort_values('r2_score_valid')

In [ ]:
df

In [ ]:
#just making it look nicer
def highlight_cols(x): 
      
    # copy df to new - original data is not changed 
    df = x.copy() 
      
    # select all values to green color 
    df.loc[:, :] = 'background-color: white and silver'
      
    # overwrite values grey color 
    df['r2_score_valid'] = 'background-color: lightgrey'
      
    # return color df 
    return df  
  
df_highlight = df.style.apply(highlight_cols, axis = None)

In [ ]:
df_highlight

In [ ]:
plt.figure(figsize=(20,20))
plt.subplots_adjust(wspace=.3,hspace=.3)

e_five =[]

for j in range(0,9):
    plt.subplot(3,3,j+1)
    e_five.append(plot_method (y_valid,e[2][j],nine_methods[j]))
    
plt.savefig('nine_ML_methods.png')

In [ ]:
plt.figure(figsize=(15,15))
plt.subplots_adjust(wspace=.3,hspace=.3)

e_five =[]

plt.subplot(2,2,1)
e_five.append(plot_method (y_valid,e[2][1],nine_methods[1]))

plt.subplot(2,2,2)
e_five.append(plot_method (y_valid,e[2][3],nine_methods[3]))

plt.subplot(2,2,3)
e_five.append(plot_method (y_valid,e[2][8],nine_methods[8]))

plt.subplot(2,2,4)
e_five.append(plot_method (y_valid,e[2][6],nine_methods[6]))
    
plt.savefig('four_ML_methods.png')

## Assingments for Orla

### 1- Add a new column with the label of the structure, for instance c11. And print with the plot with labels.


### 2- Train the model with configuration that are not in the validation set. For instance, Train with c1, c11, 12, c13, c14, c21, and use only c22 in validation set.

### 3- Work with cross-validation. PLUS: fix the "unbalacing" problem. For instance, 50 samples from each configurations.

# plot energy

In [ ]:
energy_plot = []
labels = [1, 11, 12, 13, 14, 21 ,22]
for i in labels:
    x = np.loadtxt('oszicar/energy'+str(i)+'.csv')
    energy_plot.append(x)


In [ ]:
order = [np.argsort(energy_plot[j][:,0]) for j in range(7)]


In [ ]:
x = [] 
for j in range(7):
    x.append([energy_plot[j][i,0] for i in order[j]])

In [ ]:
y = [] 
for j in range(7):
    y.append([energy_plot[j][i,1] for i in order[j]])

In [ ]:
plot_labels = ['c1', 'c11', 'c12', 'c13', 'c14','c21','c22']

In [ ]:
m5 = pd.read_csv("../../Bi6Ti5O18/ISAF-plots/BTFO-2_energy.csv")
m5.sort_values(by=['energy'])

In [ ]:
#threshold = m5.iloc[5,0]
threshold = -770.120

index = np.array(m5['config'])
delta = ((np.array(m5['energy'])-min(np.array(m5['energy'])))*1000/4)
delta1= ['+'+str("{:.2f}".format(i)) for i in delta]
c = np.array(['config. '+str(i) for i in np.array(m5['config'])])

In [ ]:
(np.array(m5['energy'])-min(np.array(m5['energy'])))

In [ ]:
fig = plt.figure()
fig.figsize=(6,6)
plt.subplots_adjust(wspace=.5,hspace=.5)
fig.patch.set_facecolor('white')
plt.plot(c,
         (np.array(m5['energy'])-min(np.array(m5['energy'])))*1000/4,
         'o')#, c="green")

plt.vlines(.5,.0,300,color="black",alpha=.7,linestyles='dashed')
plt.vlines(4.5,0,300,color="black",alpha=.7,linestyles='dashed')
#plt.hlines((threshold-min(np.array(m5['energy']))),0.0,10)

#plt.tick_params(
#    axis='y',          # changes apply to the x-axis
#    which='both', right = True,     # both major and minor ticks are affected
 #  # bottom=False,      # ticks along the bottom edge are off
    #top=False,         # ticks along the top edge are off
 #   labelright=True)

#plt.ylim(-5,120)
#plt.tick_params(axis ='both')#,labelright=True)
plt.yticks(np.arange(0.0,310, step=50),size=14)
plt.ylabel("$\Delta$ E  (meV / f.u.)" ,size=16 )
plt.xticks(rotation=45,size=14)
plt.xlabel("",size=16)
plt.title('Bi$_6$Ti$_3$Fe$_{2}$O$_{18}$ Cation Distributions',size=16)

#plt.title("LDA+U functional",size=16)
#plt.savefig('../1-year-presentation/plots/m5-ldau.png',dpi=100,edgecolor='none')
plt.savefig('B6TFO.png', dpi=600, edgecolor='none', bbox_inches = 'tight')
#plt.show()

In [ ]:
plt.figure(figsize=(8,5))
plt.subplots_adjust(wspace=.3,hspace=.3)

for i in range(7):
    n = 7-i-1
    plt.plot(x[n], y[n], '-', linewidth=2,
             markersize=20,label=labels)

plt.vlines(100,-770.5,-767,color="black",alpha=.7,linestyles='dashed')
plt.ylim(-770.5,-767.5)
plt.xlim(-10,260)
plt.legend(plot_labels[::-1],fontsize=14)
plt.ylabel(" Energy ( eV ) " ,size=16 )
plt.xlabel("Steps",size=16)
plt.title('Bi$_6$Ti$_3$Fe$_{2}$O$_{18}$ Cation Distributions',size=16)
plt.savefig('B6TFO-energy-steps.png', dpi=600, edgecolor='none', bbox_inches = 'tight')